# PILON iteration 2 & 3 for Shasta and FLYE assembly

In [1]:
WORKDIR=/workspace/hraczw/github/GA/Bilberry_genome/200.Pilon.correction/PILON_i2

In [2]:
R1=/input/genomic/plant/Vaccinium/myrtillus/Genome/AGRF_CAGRF20139_HMHTHDSXX/Clean_Billbery_HMHTHDSXX_CTAGCGCT-GTGTAGAC_L001_R1.fastq.gz
R2=/input/genomic/plant/Vaccinium/myrtillus/Genome/AGRF_CAGRF20139_HMHTHDSXX/Clean_Billbery_HMHTHDSXX_CTAGCGCT-GTGTAGAC_L001_R2.fastq.gz

## 1. PILON iteration 2

In [3]:
module load bowtie2
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bowtie2/2.3.4.3
  3) pandoc/1.19.2      6) perl/5.28.0


In [4]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assembly = line.split('\n')[0]
    assemblyName = line.split('/')[-1].split('.fasta')[0]
    os.system('bsub -J bowtie2-build \
    -n 8 \
    -o $WORKDIR/bowtie2-build_' + assemblyName + '.out \
    -e $WORKDIR/bowtie2-build_' + assemblyName + '.err \
    "bowtie2-build \
    --threads 8 ' + assembly + ' $WORKDIR/' + assemblyName + '"')
    
f.close()

EOF

Job <374733> is submitted to default queue <normal>.
Job <374734> is submitted to default queue <normal>.


In [5]:
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bowtie2/2.3.4.3
  3) pandoc/1.19.2      6) perl/5.28.0        9) samtools/1.9


In [6]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assemblyName = line.split('/')[-1].split('.fasta')[0]
    os.system('bsub -J mapping \
                -n 10 \
                -o $WORKDIR/bowtie2_' + assemblyName + '.out \
                -e $WORKDIR/bowtie2_' + assemblyName + '.err \
                "bowtie2 \
                -p 10 \
                -x $WORKDIR/' + assemblyName + ' \
                -1 $R1 \
                -2 $R2 | \
                samtools view -bS - > $WORKDIR/' + assemblyName + '.bam"')


EOF

Job <374735> is submitted to default queue <normal>.
Job <374736> is submitted to default queue <normal>.


In [7]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assemblyName = line.split('/')[-1].split('.fasta')[0]

    os.system('bsub -J sorting \
    -n 10 \
    -o $WORKDIR/sort_' + assemblyName + '.out \
    -e $WORKDIR/sort_' + assemblyName + '.err \
    "samtools sort \
    -@ 10 \
    -o $WORKDIR/' + assemblyName + '.sorted.bam \
    $WORKDIR/' + assemblyName + '.bam"')


EOF

Job <374961> is submitted to default queue <normal>.
Job <374962> is submitted to default queue <normal>.


In [8]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assemblyName = line.split('/')[-1].split('.fasta')[0]

    os.system('bsub -J indexing \
    -o $WORKDIR/index_' + assemblyName + '.out \
    -e $WORKDIR/index_' + assemblyName + '.err \
    "samtools index $WORKDIR/' + assemblyName + '.sorted.bam"')


EOF

Job <374979> is submitted to default queue <normal>.
Job <374980> is submitted to default queue <normal>.


In [9]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assembly = line.split('\n')[0]
    assemblyName = line.split('/')[-1].split('.fasta')[0]

    os.system('bsub -J pilon \
    -m wkoppb50 \
    -n 60 \
    -o $WORKDIR/' + assemblyName + '_pilon-i2.out \
    -e $WORKDIR/' + assemblyName + '_pilon-i2.err \
    "taskset -a -c 0-60 java -Xmx1500G -jar /software/bioinformatics/pilon-1.23/pilon-1.23.jar \
     --genome ' + assembly + ' \
     --frags $WORKDIR/' + assemblyName + '.sorted.bam \
     --output ' + assemblyName + '_pilon-i2_corrected \
     --outdir $WORKDIR/ \
     --changes \
     --diploid \
     --fix all \
     --threads 60 \
     --flank 5"')
     
f.close()

EOF

Job <374987> is submitted to default queue <normal>.
Job <374988> is submitted to default queue <normal>.


## 2. PILION iteration 3

In [17]:
WORKDIR=/workspace/hraczw/github/GA/Bilberry_genome/200.Pilon.correction/PILON_i3

In [18]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assembly = line.split('\n')[0]
    assemblyName = line.split('/')[-1].split('.fasta')[0]
    os.system('bsub -J bowtie2-build \
    -n 16 \
    -o $WORKDIR/bowtie2-build_' + assemblyName + '.out \
    -e $WORKDIR/bowtie2-build_' + assemblyName + '.err \
    "bowtie2-build \
    --threads 16 ' + assembly + ' $WORKDIR/' + assemblyName + '"')
    
f.close()

EOF

Job <376322> is submitted to default queue <normal>.
Job <376323> is submitted to default queue <normal>.


In [19]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assemblyName = line.split('/')[-1].split('.fasta')[0]
    os.system('bsub -J mapping \
                -n 20 \
                -o $WORKDIR/bowtie2_' + assemblyName + '.out \
                -e $WORKDIR/bowtie2_' + assemblyName + '.err \
                "bowtie2 \
                -p 20 \
                -x $WORKDIR/' + assemblyName + ' \
                -1 $R1 \
                -2 $R2 | \
                samtools view -bS - > $WORKDIR/' + assemblyName + '.bam"')


EOF

Job <376324> is submitted to default queue <normal>.
Job <376325> is submitted to default queue <normal>.


In [20]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assemblyName = line.split('/')[-1].split('.fasta')[0]

    os.system('bsub -J sorting \
    -n 10 \
    -o $WORKDIR/sort_' + assemblyName + '.out \
    -e $WORKDIR/sort_' + assemblyName + '.err \
    "samtools sort \
    -@ 10 \
    -o $WORKDIR/' + assemblyName + '.sorted.bam \
    $WORKDIR/' + assemblyName + '.bam"')


EOF

Job <376491> is submitted to default queue <normal>.
Job <376492> is submitted to default queue <normal>.


In [21]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assemblyName = line.split('/')[-1].split('.fasta')[0]

    os.system('bsub -J indexing \
    -o $WORKDIR/index_' + assemblyName + '.out \
    -e $WORKDIR/index_' + assemblyName + '.err \
    "samtools index $WORKDIR/' + assemblyName + '.sorted.bam"')


EOF

Job <376518> is submitted to default queue <normal>.
Job <376519> is submitted to default queue <normal>.


In [22]:
python << EOF

import sys, os

f = open('$WORKDIR/assemblies.txt', 'r')

for line in f:
    assembly = line.split('\n')[0]
    assemblyName = line.split('/')[-1].split('.fasta')[0]

    os.system('bsub -J pilon \
    -m wkoppb50 \
    -n 60 \
    -o $WORKDIR/' + assemblyName + '_pilon-i3.out \
    -e $WORKDIR/' + assemblyName + '_pilon-i3.err \
    "taskset -a -c 0-60 java -Xmx1500G -jar /software/bioinformatics/pilon-1.23/pilon-1.23.jar \
     --genome ' + assembly + ' \
     --frags $WORKDIR/' + assemblyName + '.sorted.bam \
     --output ' + assemblyName + '_pilon-i3_corrected \
     --outdir $WORKDIR/ \
     --changes \
     --diploid \
     --fix all \
     --threads 60 \
     --flank 5"')
     
f.close()

EOF

Job <376528> is submitted to default queue <normal>.
Job <376529> is submitted to default queue <normal>.
